In [19]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np 
from multiprocessing import Pool
import time 
import random
import IPython.display as ipd
import pyaudio
import sys
if '../../' not in sys.path: sys.path.insert(1, "../../")

#import utils.audio as a
#import utils.plots as plots
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:

print(sys.path)

['/home/tamay/Projects/git/laughter-synthesis/synthesizer/full_audio', '../../', '/opt/anaconda3/envs/aip37/lib/python37.zip', '/opt/anaconda3/envs/aip37/lib/python3.7', '/opt/anaconda3/envs/aip37/lib/python3.7/lib-dynload', '', '/home/tamay/.local/lib/python3.7/site-packages', '/opt/anaconda3/envs/aip37/lib/python3.7/site-packages', '/home/tamay/Projects/sentiment-analysis/laughter/scripts/wavenet_vocoder', '/opt/anaconda3/envs/aip37/lib/python3.7/site-packages/IPython/extensions', '/home/tamay/.ipython']


In [2]:
# load (fe)male audio sequences without alpha, beta 
female_audios = np.load("female_audios/female_audios_500.npy")
male_audios = np.load("male_audios/male_audios_500.npy")

N = 20
ratio = 0
n_male = int(ratio * N)
n_female = N - n_male

In [47]:
female_audios.shape[0]

500

In [3]:
# get male indices and audio signals
male_idx = np.random.choice(male_audios.shape[0], n_male)
males = [male_audios[i] for i in male_idx]

# get female indices and audio signals
female_idx = np.random.choice(female_audios.shape[0], n_female)
females = [female_audios[i] for i in female_idx]

# concatenate both lists and shuffle
mixed_audio = np.array(random.sample(males+females, N))

# create list of alpha values for stereo
alpha=np.random.rand(N)
# create list for beta fallof
beta=np.reciprocal(np.sqrt(np.arange(1, N+1)))

In [4]:
# synthesize left audio
left_audio = N * alpha[:, np.newaxis] * beta[:, np.newaxis] * mixed_audio
# synthesize right audio
right_audio = N * (1-alpha[:, np.newaxis]) * beta[:, np.newaxis] * mixed_audio
# concatenate and compute mean
audio = np.array([left_audio.mean(0), right_audio.mean(0)])

print(audio.shape)

(2, 220500)


In [5]:
SR = 22050 
# 1000
ipd.Audio(audio, rate=SR) # load a NumPy array

In [22]:
SR = 22050
#10
ipd.Audio(audio, rate=SR) 

In [27]:
SR = 22050
#10
ipd.Audio(audio, rate=SR)

In [ ]:
amount of persons =  

female/male distribution
clapping 0-100
whistling 0-100
laughter = 0-100
enthusiasm = 100
booing = 0-100

In [6]:
CHUNK = 1024
FORMAT = pyaudio.paFloat32
CHANNELS = 1
SR = 22050
RECORD_SECONDS = 10
# instantiate PyAudio (1)
p = pyaudio.PyAudio()

# open stream (2)
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=SR,
                output=True)

# read data
data = utils.misc.slidingWindow(audio.mean(0), int(SR*1), int(SR*1))

# play stream (3)
for chunk in data:
    print(chunk.shape)
    stream.write(chunk.astype(np.float32).tostring())

# stop stream (4)
stream.stop_stream()
stream.close()

# close PyAudio (5)
p.terminate()

NameError: name 'utils' is not defined

In [7]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import math
%matplotlib inline

In [8]:
import pyaudio
soundObj = pyaudio.PyAudio()

# Learn what your OS+Hardware can do
defaultCapability = soundObj.get_default_host_api_info()
print(defaultCapability)

# See if you can make it do what you want
isSupported = soundObj.is_format_supported(input_format=pyaudio.paFloat32, input_channels=2, rate=44100, input_device=0)
print(isSupported)

{'index': 0, 'structVersion': 1, 'type': 8, 'name': 'ALSA', 'deviceCount': 16, 'defaultInputDevice': 15, 'defaultOutputDevice': 15}
True


In [ ]:
from ipywidgets import Layout, Button, Box, HBox
import threading
from IPython.display import display
import ipywidgets as widgets
import time
import math
import pdb 
import scipy


SR = 44100#22050
#upsample audio
print(audio.shape)
audio_up = scipy.signal.resample(audio, 2*audio.shape[1], axis=1)

print(audio_up.shape)
CHUNK_SEC = 1
CHUNK = int(CHUNK_SEC * SR)
FORMAT = pyaudio.paFloat32
CHANNELS = 2
RECORD_SECONDS = 10

p = pyaudio.PyAudio()
# open stream (2)
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=SR,
                output=True)

progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
total = 100
run = True
stop = False

# instantiate PyAudio (1)
#data = sliding_window(audio_up, CHUNK, CHUNK).transpose(1, 2, 0)


ratio_slider = widgets.FloatSlider(
    value=.50,
    min=0.,
    max=1000.0,
    step=1.,
    description='Female/Male Ratio:',
    readout_format='.1f',
)

button_start = widgets.Button(
    description='Start',
    icon='check'
)

button_stop = widgets.Button(
    description='Stop',
    icon='check'
)


output_start = widgets.Output()
output_stop = widgets.Output()        
output_slider = widgets.Output()

run = False
stop = False


def work(progress):
    global stop, stream
    i = 0
    stream.start_stream()
    while not stop:
        if run:
            progress.value = float((i*10))/total
            stream.write(data[i].astype(np.float32).tostring())
            i += 1
            i = i%10
            time.sleep(0.01)

        print("Thread terminated")

def on_button_clicked_start(b):
    with output_start:
        run = True
        
def on_button_clicked_stop(b):
    global stop
    with output_stop:
        #stream.stop_stream()
        stop = True
        thread.join() 
        print("Stream stopped")

def on_value_change(change):
    global total
    with output_slider:
        print(change['new'])
        total = change['new']

ratio_slider.observe(on_value_change, names='value')
button_start.on_click(on_button_clicked_start)
button_stop.on_click(on_button_clicked_stop)

display(button_start, output_start)
display(button_stop, output_stop)
display(ratio_slider, output_slider)

#items = [button_start_pause, button_stop, ratio_slider]
#box_layout = Layout(display='flex', flex_flow='row',
#                    align_items='flex-start', width='100%')
#HBox(items, layout=box_layout)

try:
    thread = threading.Thread(target=work, args=(progress,))
    display(progress)
    thread.start()
finally:
    if stream.is_active(): 
        stream.stop_stream()
    stream.close()
    p.terminate()
    thread.join()

In [10]:

ratio_slider = widgets.FloatSlider(
    value=.50,
    min=0.,
    max=1000.0,
    step=1.,
    description='Female/Male Ratio:',
    readout_format='.1f',
)

button_start = widgets.Button(
    description='Start',
    icon='check'
)

button_stop = widgets.Button(
    description='Stop',
    icon='check'
)


output_start = widgets.Output()
output_stop = widgets.Output()        
output_slider = widgets.Output()

def on_button_clicked_start(b):
    global run
    with output_start:
        run = True
        
def on_button_clicked_stop(b):
    global stop
    with output_stop:
        #stream.stop_stream()
        run = False
        print("Stream stopped")

def on_value_change(change):
    global total
    with output_slider:
        print(change['new'])
        total = change['new']

ratio_slider.observe(on_value_change, names='value')
button_start.on_click(on_button_clicked_start)
button_stop.on_click(on_button_clicked_stop)

display(button_start, output_start)
display(button_stop, output_stop)
display(ratio_slider, output_slider)

Button(description='Start', icon='check', style=ButtonStyle())

Output()

Button(description='Stop', icon='check', style=ButtonStyle())

Output()

FloatSlider(value=0.5, description='Female/Male Ratio:', max=1000.0, readout_format='.1f', step=1.0)

Output()

In [11]:
from ipywidgets import Layout, Button, Box, HBox
import threading
from IPython.display import display
import ipywidgets as widgets
import time
import math
import pdb 
import scipy

SR = 44100#22050
print(audio.shape)
audio_up = scipy.signal.resample(audio, 2*audio.shape[1], axis=1)

print(audio_up.shape)
CHUNK_SEC = 1
CHUNK = int(CHUNK_SEC * SR)
FORMAT = pyaudio.paFloat32
CHANNELS = 2
RECORD_SECONDS = 10
run = False
stop = False

p = pyaudio.PyAudio()
# open stream (2)
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=SR,
                output=True)

progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
total = 100
run = False
stop = False

data = utils.misc.slidingWindow(audio_up, CHUNK, CHUNK).transpose(1, 2, 0)

def work(progress):
    global stop, stream
    i = 0
    stream.start_stream()
    while not stop:
        if run:
            progress.value = float((i*10))/total
            stream.write(data[i].astype(np.float32).tostring())
            i += 1
            i = i%10
            time.sleep(0.01)
        

thread = threading.Thread(target=work, args=(progress,))
display(progress)
thread.start()

(2, 220500)
(2, 441000)


FloatProgress(value=0.0, max=1.0)

In [ ]:
stop = True
if stream.is_active(): 
    stream.stop_stream()
stream.close()
p.terminate()
thread.join()

### Code snippet for mel spectrogram

In [ ]:
# manual computation
n_fft=1024
hop_length=256
win_length=1024
sampling_rate=22050
n_mel_channels=80
mel_fmin=0.0
mel_fmax=None

audio, sr = librosa.load(file)
audio = librosa.util.fix_length(audio, sampling_rate * 10) # fix the audio file to 10 seconds crop or pad with zeros

#x = audio
#audio = (x-min(x))/(max(x)-min(x))

window = torch.hann_window(win_length).float()
mel_basis = librosa_mel_fn(sampling_rate, n_fft, n_mel_channels, mel_fmin, mel_fmax)
mel_basis = torch.from_numpy(mel_basis).float()
audio = torch.from_numpy(audio)[None]


p = (n_fft - hop_length) // 2
audio = F.pad(audio, (p, p), "constant").squeeze(1)
fft = torch.stft(
        audio,
        n_fft=n_fft,
        hop_length=hop_length,
        win_length=win_length,
        window=window,
        center=False,
    )
real_part, imag_part = fft.unbind(-1)
magnitude = torch.sqrt(real_part ** 2 + imag_part ** 2)
mel_output = torch.matmul(mel_basis, magnitude)
log_mel_spec = torch.log10(torch.clamp(mel_output, min=1e-5))


# librosa
fccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40) #has quite a bunch of parameters